In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

In [21]:
! pip install simplet5 textract -q

     |████████████████████████████████| 32.8 MB 1.6 MB/s 
     |████████████████████████████████| 5.6 MB 20.8 MB/s 
     |████████████████████████████████| 10.1 MB 19.2 MB/s 
     |████████████████████████████████| 103 kB 48.3 MB/s 
     |████████████████████████████████| 69 kB 5.6 MB/s 
     |████████████████████████████████| 106 kB 50.8 MB/s 
     |████████████████████████████████| 2.0 MB 29.8 MB/s 
     |████████████████████████████████| 128 kB 51.1 MB/s 
     |████████████████████████████████| 112 kB 31.5 MB/s 
     |████████████████████████████████| 73 kB 2.4 MB/s 
     |████████████████████████████████| 149 kB 29.8 MB/s 
     |████████████████████████████████| 70 kB 5.9 MB/s 
     |████████████████████████████████| 339 kB 47.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which i

# Read Input Dataset - desired columns `headlines` & `text`

In [8]:
df = pd.read_csv("/content/Summary-dataset - db1.csv",usecols=['highlights', 'article'])
import re
df['article'] = df['article'].apply(lambda x: re.sub('\n'," ",x))
df['highlights'] = df['highlights'].apply(lambda x: re.sub('\n'," ",x))

In [9]:
df.head()

,article,highlights
0,Core Qualifications Drafting Space planning In...,Interior Designer with a distinct understandin...
1,Interior Designer with a distinct understandin...,Passionate personwith two years ofexperienceas...
2,The Mission ~ To work with ateamas designer or...,My career of 34 years includes the graphicart ...
3,Experience Freelance UX/UI InteractionDesigner...,Combined withwork experienceinArchitecture to ...
4,"Highlights Adobe Creative Suite(Illustrator, P...","Agraphic designer, who iscreativeand detail-or..."


# T5 Data Prep with Training Data Column Names - `source_text` & `target_text`

In [11]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"highlights":"target_text", "article":"source_text"})
df = df[['source_text', 'target_text']]


In [12]:
df.head()

,source_text,target_text
0,Core Qualifications Drafting Space planning In...,Interior Designer with a distinct understandin...
1,Interior Designer with a distinct understandin...,Passionate personwith two years ofexperienceas...
2,The Mission ~ To work with ateamas designer or...,My career of 34 years includes the graphicart ...
3,Experience Freelance UX/UI InteractionDesigner...,Combined withwork experienceinArchitecture to ...
4,"Highlights Adobe Creative Suite(Illustrator, P...","Agraphic designer, who iscreativeand detail-or..."


# T5 Data Prep with Summarization Tax Prefix

In [13]:
# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,summarize: Core Qualifications Drafting Space ...,Interior Designer with a distinct understandin...
1,summarize: Interior Designer with a distinct u...,Passionate personwith two years ofexperienceas...
2,summarize: The Mission ~ To work with ateamas ...,My career of 34 years includes the graphicart ...
3,summarize: Experience Freelance UX/UI Interact...,Combined withwork experienceinArchitecture to ...
4,summarize: Highlights Adobe Creative Suite(Ill...,"Agraphic designer, who iscreativeand detail-or..."
...,...,...
693,summarize: Highlights\r Businessanalysis\r Onl...,Sr. E-commerce Merchandiser specializing in au...
694,"summarize: Highlights\r Ms-Office(Excel, Word,...",To develop and find a placein theindustrywhere...
695,"summarize: Core Qualifications\r MS Outlook, W...",To applymyselfin a newand challenging position...
696,summarize: Experience\r Substitute Teacher\r D...,Seeking anAdministrative positionwhere my prof...


In [14]:
train_df, test_df = train_test_split(df, test_size=0.3)
train_df.shape, test_df.shape

((488, 2), (210, 2))

# Using SimpleT5 for Model Training - Instantiate, Download Pre-trained Model

In [15]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")


Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

# Model Training

In [16]:
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100], 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=5, use_gpu=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Output Folder Content


In [17]:
! ( cd outputs; ls )

simplet5-epoch-0-train-loss-4.0566-val-loss-3.5542
simplet5-epoch-1-train-loss-3.5995-val-loss-3.4056
simplet5-epoch-2-train-loss-3.3338-val-loss-3.3061
simplet5-epoch-3-train-loss-3.1178-val-loss-3.232
simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214


# Model Inference

In [19]:
# let's load the trained model from the local output folder for inferencing:
model.load_model("t5-base","simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214", use_gpu=True)



# Impressive Results

In [39]:
import textract

data=textract.process('/content/1088.pdf')
data1 = re.sub('\n'," ",str(data))
data1 = re.sub('\\n'," ",str(data1))

In [40]:
data1

"b'SANJAYSINH  JADEJA\\n\\nVILLAGE\\nTALUKA\\nDIST\\n\\nMob.\\nE-mail\\n            \\n\\n:\\n:\\n:\\n\\n:\\n:\\n\\nBEHIND POST OFFICE,\\nMOTAKAPADA,\\nDISA.\\nBANASKATHA\\nGUJARAT\\n+91 9909545802\\nsanjay4832@gmail.com\\n \\n\\n  / sanjay4832@yahoo.co.in\\n \\n \\n\\n APPLICATION\\n\\nSub: Application for the vacant post stores in your esteemed \\n\\n .  \\n Organization\\n \\n\\n \\n\\nRespected Sir,\\n\\nI have come to know from some reliable sources that there is a vacancy in your reputed organization for\\n\\nabove position. I would like to apply my self for it.\\n\\nI have total 13 years experience store dept. of various construction companies. I will submit my credentials while\\nreceiving your reply for attending the interview.\\n\\nI hope you will consider my application sympathically and looking forward for favorable, positive response at\\nearliest convinced same as per requirement for the position.\\n\\nThanking you in anticipation.\\n\\nYours sincerely,\\n\\n(Jadeja Sanja

In [33]:
text_to_summarize="""summarize:b'Supriya Yadav \nContact No.: - 8828188400 \nE-mail:-supriyadav0562@gmail.com \n \nPresent Address:- Room No-591/002, Shiravane Village, Sector-1,Nerul,New \n\nMumbai-400706. \n\n \n \n \n \n \n \n \n\n \n \nCAREER OBJECTIVE \n \nTo work in a firm with a professional work driven environment where I can utilize and apply my \nknowledge, skills which would enable me as a fresh graduate to grow while fulfilling organizational \ngoals. \nProficient  in  Finding  and  Resolving  Malfunctions,  Using  Exceptional  Technical  and  Communication \nSkills. Ability to Express Technical Concepts Clearly To People with No Technical Background.  \n \nBASIC ACADEMIC CREDENTIALS  \n\nQualification \n\nSchool/College Name \n\nBoard/University  Year  Percentage/\n\nBSc.(IT) \n\nPillai College of Arts, Commerce & \nScience \n\nMumbai University \n\n2021 \n\n81.70% \n\nCGPI \n\nP.V.G. Vidya Bhawan High School \n& Jr. College \n\nMaharashtra State \nBoard \n\nSt. Xavier\xe2\x80\x99s High School & Jr. \nCollege \n\nMaharashtra State \nBoard \n\n2018 \n\n62.70 % \n\n2016 \n\n82.20% \n\nHSC \n\nSSC \n\n \n\n \n\n \nIT PROFICIENCY \n \n\n \n\n\xe2\x9d\x96  Languages \n\xe2\x9d\x96   Web Programming/Scripting \n\xe2\x9d\x96   DATABASE \n\xe2\x9d\x96   Operating System \n\n: JAVA,PYTHON \n: HTML,CSS,JAVASCRIPT \n: MySql \n: Microsoft Windows. \n\n \n \nINTERPERSONAL SKILL \n\n\xe2\x9d\x96  Ability to rapidly build relationship and set up trust. \n\xe2\x9d\x96  Confident and Determined \n\xe2\x9d\x96  Ability to cope up with different situations. \n\xe2\x9d\x96  Good communication skill. \n\xe2\x9d\x96  Positive attitude Towards Work and Self Motivated. \n\xe2\x9d\x96  Quick Learner. \n\n \n\n \n \n \n\n \n\n \n \n \n \n\n\x0cCERTIFICATION  \n \n\xe2\x9d\x96  Coursera Certification for Python \n\xe2\x9d\x96  Digital Marketing Google Certification \n\xe2\x9d\x96  Google  Ad\xe2\x80\x99s  Certification \n \n\n \n\nPERSONAL DETAILS \n \n \n\n\xe2\x9d\x96  Date of Birth      \n\xe2\x9d\x96  Language Known \n\xe2\x9d\x96  Marital Status   \n\xe2\x9d\x96  Nationality \n \n \n\n \n \n \n \n\n:-   5th June 2000 \n:-   English, Hindi and Marathi \n:-   Unmarried \n:-   Indian  \n\n \nDECLARATION  \n \nI do hereby declare that the above information is true to the best of my knowledge. \n \n \n\nPlace:-  Nerul, Navi Mumbai  \nDate:-  \n \n\n \n\n \n\n \n \n\n                 SUPRIYA YADAV (Signature) \n\n \n\n \n \n\n \n \n\n \n \n \n\n\x0c'  """
model.predict(text_to_summarize)

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors


["'Supriya Yadav' is an IT professional with over 20 years of experience in the field of software development and implementation. I have been involved in various projects, both small and large, from early stages of my career to completion."]

In [36]:
text_to_summarize="""summarize: b'SUKANYA RAMESH\n\nMCA\n\nNo :501,BUILDING NO:16 ,\n\nTILAK VAIBHAV CHS,\n\nSAINATH MANDIR CHOWK, \n\nTILAK NAGAR,\n\nchembur, mumbai- 400 089\n\nsugan882@gmail.COM\n\n Mobile no  +91 8369541044\n\nSEP 2014\n\n\n\nKey Skills\n\n\n\nSelenium\n\nTestNG\n\nSOAP  UI\n\nBDD Framework\n\nTest Management\n\nAgile Test Practices\n\nRelease Coordination\n\nTeam Mentoring\n\nVendor Management\n\n\n\nOBJECTIVE\n\nA versatile and enthusiastic software test lead seeking challenges to learn and grow.\n\n\n\nSUMMARY\n\n\n\nExperience: 10.6 years.\n\nDomains Worked: Health Care, Asset Management,  Reference Data-Client, International Wealth Management\n\n\n\nEXPERIENCE\n\n\n\nMORGAN STANLEY ADVANTAGE SERVICES, Mumbai, India\n\n\n\nSTP- Order Management System\n\n\n\nSenior Manager,                                                                                                    May  2018 - Present\n\n\n\nProject Details\n\nThis is a Order Management application for placing the order for different kind of assets mainly on Asia Market. The System takes the trade through the trade lifecycle to book the order. STP communicates through web services to various Upstream and downstream systems to validate the client, products traded, client position, suitability and eligibility checks to complete the trade.\n\nAlso, the system calculates the commission charges for the broker account based on the type of order and communicates with exchange for OTC products.\n\nApart from the trades, the project also includes Runtime Trade Checks(RTC) and Run The Bank(RTB) enhancements across securities and client positions.\n\n\n\nTools Used:  Selenium, BDD Cucumber Framework, Test Management-JIRA,  Agile Test Practices, SOAP UI, Vendor Management, Team mentoring,\n\n\n\n\n\nJP MORGAN India Pvt Ltd., Mumbai, India\n\n\n\nWCOB Applications - Client Due Diligence MORCOM, CFA, FATCA\n\n\n\nTest Lead, \t                                                                                                            Aug  2013 - July 2016\n\n\n\nProject Details\n\nThis is a Reference Data application for the Client Onboarding  as per Anti Money Laundering(AML) regulatory system for Investment Banking Products. The Client Due Diligence(CDD) is used to send KYC status and AML report to various downstream systems. CDD also manages and updates the various stages of KYC status immediately to downstream systems on a intraday. \n\n\n\n\n\nTools Used:  Selenium, TestNG, TDD framework, JIRA,  MULE MQ, Control-M job as scheduler,  Interfaces, SOAP UI.\n\n\n\nMiddle Office and Front Office Tools- CMA, CAS, OSIRIS, OES\n\n\n\nQA Analyst, \t                                                                                                            May 2010 \xe2\x80\x93 July 2013\n\n\n\nProject Details\n\nA banking application for the cash manager/portfolio manager for managing and overseeing the cash balances within the universe of Institutional accounts whose daily activities include: monitoring balances and Overdrafts, Raising Orders for Cash Trades (on the basis of account guidelines or on instruction from the relevant Portfolio Manager), Posting Client Cash Flows to the account, Controls and investigation of Reconciliation Breaks, Real World Payments, Payment Requests and Reporting.\n\n\n\nTools Used:  ALM- QC, JIRA, MULE MQ queue , Bloomberg for rates, SOAP UI, QTP.\n\n\t\t\n\n\n\nHewitt Associates India Pvt Ltd, Chennai, India\n\n\n\nAnnual Enrollment, Prices and Premium Processing, Dependent Audit and Certification\n\nClients: TDS, Mead Westvaco, Vision Inc, Bayer Corporation, USA\n\n\n\nSr. Configuration Analyst, \t                                                                         Dec 2007 \xe2\x80\x93 Jan 2010\n\n\n\nProject Details\n\nA Human Resource Benefits application to create and renew changes to their Health and  Group Plans, calculate premiums based on their pay rate, pay schedule and auditing of the all the employee(s) specific dependent(s) details. Calculation of premium for potential dependents of the employees eligible for coverage. \n\n\n\nCobra Personal Assistance Program and Cobra Processing \n\nClients- General Mills, Ameren Corporation, USA\n\n\n\nConfiguration Analyst, \t                                                                                Feb 2006 \xe2\x80\x93 Jan 2008\n\n\n\nProject Details\n\nA Human Resource Benefits application to create and renew changes to Cobra benefits based on the age and length of service with the company subsidy code provided by Clients. Cobra employees are eligible for the Health Plans (Medical, Dental, Vision). Actions include potential dependents of the employees eligible for coverage by Age and Disability.\n\n\n\n\n\n\n\n\n\nSukanya Ramesh.'"""
model.predict(text_to_summarize)

["'Successful' personable, highly skilled and self-motivated IT professional with more than 20 years of experience in the field of Human Resources, Health Care and Disability Services. Proven track record of successfully completing multiple projects simultaneously while maintaining high standards of quality."]

In [41]:
text_to_summarize="""summarize: b'                                      CURRICULAM VITE\n                                      \n\nCURRICULAM VITE            \n            \n\n     \n     \nNAME:-TANZEER AHMAD SHADAB\nNAME:-TANZEER AHMAD SHADAB\nContact No:-+91-7488305425\nContact No:-+91-7488305425\nEmail:Address:-tanzeer204@gmail.com\ntanzeer204@gmail.com    \nEmail:Address:-\n\nSubject: Storekeeper & Warehouse/stock In charge    \nSubject: Storekeeper & Warehouse/stock In charge\nApplication for The Post of Storekeeper & Wareouse In Charge    \nApplication for The Post of Storekeeper & Wareouse In Charge\nObjective:    \nObjective:\nTo pursue a challenging team career in the field of construction/Electrical\nTo pursue a challenging team career in the field of construction/\ncan utilize my skills, innovative ideas & ready to face new challenges.\ncan utilize my skills, innovative ideas & ready to face new challenges.\nBrief Overview:    \nBrief Overview:\nSeasoned Storekeeper with expertise in warehousing that exercise general control over all activities in \nSeasoned Storekeeper with expertise in warehousing that exercise general control over all activities in \nKnowledgeable in ERP & GP applications and maintaince of proper records.Initiates \nstores department. Knowledgeable in ERP & GP\n applications and maintaince of proper records.Initiates \nstores department. \nmaterial requests coming for different Electrical /construction site and timely follow-up of purchase \nmaterial requests coming for different Electrical /construction site and timely follow-up of purchase \norder from different suppliers in order making sure that there are adequate stock in the store. \norder from different suppliers in order making sure that there are adequate stock in the store. \nConducts actual physical stock inventory.\nConducts actual physical stock inventory.\n\nElectrical Business as a \xe2\x80\x98storekeeper\xe2\x80\x99, I \n Business as a \xe2\x80\x98storekeeper\xe2\x80\x99, I \n\nMy Strengths:    \nMy Strengths:\n\n11-Friendly, outgoing person adapts easily to various situations.\n-Friendly, outgoing person adapts easily to various situations.\n22-Open to learn new skills and willing to work long hours, as required.\n-Open to learn new skills and willing to work long hours, as required.\n33-Good sense of humor.\n-Good sense of humor.\n44-Enjoy problem solving and taking challenges\n-Enjoy problem solving and taking challenges\n55-Results oriented.\n-Results oriented.\nWork Experience:    \nWork Experience:\n                             4 YEARS EXPERIENCE SAUDI ARABIA (K.S.A)\n4 YEARS EXPERIENCE SAUDI ARABIA (K.S.A)\n                             \n                              AL GIHAZ CO.LTD (CONTRACTING TRADING&TOURISM)\nAL GIHAZ CO.LTD (CONTRACTING TRADING&TOURISM)  \n                              \nBrief Description of Project    \nBrief Description of Project\n\n1-PROJECT          : TABUK-11 110 13.8 KB ELECTRICAL SUBSTATIONS-K.S.A\n1-PROJECT          : TABUK-11 110 13.8 KB ELECTRICAL SUBSTATIONS-K.S.A\n2-PROJECT          : TABUK 380 BSP ELECTRICAL SUBSTATIONS-K.S.A\n2-PROJECT          : TABUK 380 BSP ELECTRICAL SUBSTATIONS-K.S.A\n3-PROJECT          :  JEDDAH AL RAWDAH 110, 13.8 KV ELECTRICAL SUBSTAION-K.S.A\n3-PROJECT          :  JEDDAH AL RAWDAH 110, 13.8 KV ELECTRICAL SUBSTAION-K.S.A\n4-PROJECT          : JEDDAH EXT.KHULAISH 380 ELECTRICAL SUBSTATION-K.S.A\n4-PROJECT          : JEDDAH EXT.KHULAISH 380 ELECTRICAL SUBSTATION-K.S.A\nPOSITION             : STOREKEEPER & WAREHOUSE IN CHARGE\nPOSITION             : STOREKEEPER & WAREHOUSE IN CHARGE\nDURATION            : 2 OCTOBER 2013 TO 2 OCTOBER 2017 \nDURATION            : 2 OCTOBER 2013 TO 2 OCTOBER 2017 \n\nDuties & Responsibilities:1:- Firstly checking the documents,invoice,delivery note,packing lists And the materials \n1:- Firstly checking the documents,invoice,delivery note,packing lists And the materials \nDuties & Responsibilities:\nquantity         verified with p.o quantity then after unloading the materials at site,After Then Informing QC \nquantity         verified with p.o quantity then after unloading the materials at site,After Then Informing QC \nDepartment for taking sample for analysis after approve materials by QC Department, \nDepartment for taking sample for analysis after approve materials by QC Department, \n\n2:- All Receipt/issue& inspection of materials in store along with proper formalities & documentations \n2:- All Receipt/issue& inspection of materials in store along with proper formalities & documentations \n3:- Responsible for the store,The materials & the manpower associated with it, including its safe custody & security.\n3:- Responsible for the store,The materials & the manpower associated with it, including its safe custody & security.\n\n\x0c4:- Daily Materials stock register maintains.\n4:- Daily Materials stock register maintains.\n5:- Daily site issue Register Maintains with software entry system.\n5:- Daily site issue Register Maintains with software entry system.\n6:- Daily basis receipt, issued & balance maintain to be system\n6:- Daily basis receipt, issued & balance maintain to be system\n7:- Co-ordinate to be site Engineer & site supervisor.\n7:- Co-ordinate to be site Engineer & site supervisor.\n8:- Maintain the warehpouse, record area and stores area in a neat & orderly fashion.\n8:- Maintain the warehpouse, record area and stores area in a neat & orderly fashion.\n9:- Daily & monthly reports of consumable/materials status\n9:- Daily & monthly reports of consumable/materials status\n10:- Issuing materials and tools need of the worker.\n10:- Issuing materials and tools need of the worker.\n11:- Accountable tools issued to all worker.\n11:- Accountable tools issued to all worker.\n12:- File maintains for all outgoing & incoming materials.\n12:- File maintains for all outgoing & incoming materials.\n13:- scrap materials arrange in their corresponding locations.\n13:- scrap materials arrange in their corresponding locations.\n14:-   I   handle   tools   &   materials   at   site(GIS,SWITCHGEAR,CONTROL   PANEL,   LIGHTING   MATERIALS   AND   CIVIL\nGIS,SWITCHGEAR,CONTROL   PANEL,   LIGHTING   MATERIALS   AND   CIVIL\n14:-   I   handle   tools   &   materials   at   site(\nMATERIALS.    \nMATERIALS.\n\n(1years diploma)\n Safety Officer Certified(1years diploma)\n\nS Word, M MS S AcceAccess,ss,\n\nsonal Profile \nProfile     \n\n: Oracle Software-ERRPP&&GPGP    \n\n                                : Oracle Software-E\n\nS Excel, Internet Explorer, Outlook,                         \n                         \n\n         : H.S.E Courses Degree .1- (IOSH) Managing Safely)\n H.S.E Courses Degree .1- (IOSH) Managing Safely)\n\n : Graduate from Ranchi University in arts system(B.A)\nEDUCATIONAL QUALIFICATION : Graduate from Ranchi University in arts system(B.A)\nEDUCATIONAL QUALIFICATION\n: Intermediate from Jharkhand academic council Ranchi in Science\n                                                          : Intermediate from Jharkhand academic council Ranchi in Science\n                                                          \n: Matric borard from JSEB Ranchi-10 MAY 2003\n                                                          : Matric borard from JSEB Ranchi-10 MAY 2003\n                                                          \n: Microsoft office xp, Microsoft office 2000, MS Word,\n                             : Microsoft office xp, Microsoft office 2000, M\nComputer SkillSkillss                             \nComputer \n                                                            MMS Excel, Internet Explorer, Outlook,\n                                                            \nSoftware Skill                                \nSoftware Skill\nProfessional Qualification         :\nProfessional Qualification\n                                                                                              2-2- Safety Officer Certified\n                                                                                              \nPerPersonal \nName                                                : Tanzeer Ahmad Shadab\nName                                                : Tanzeer Ahmad Shadab\nFather Name                                   : Shabbir Ahmad\nName                                   : Shabbir Ahmad\nFather \nGender                                             : Male \nGender                                             : Male \nLanguage Known                           : English, Urdu, \nLanguage Known                           : Engli\nPermanent Address                       \nPermanent Addre\nDate Of Birth                                   : 12-12-1988\nBirth                                   : 12-12-1988\nDate Of \nMarital Status                                  : \nMarital Statu\nNationality                                       : Indian\nNationality                                       : Indian\nReligion                                            : Islamslam\nReligion                                            : I\nNo                                     : J9801861\nPaPassport \nssport No                                     : J9801861\nPlace of Issue                                  :RANCHI\nssue                                  :RANCHI\nPlace of I\nPlace of Birth                                  :RANCHI JHARKHAND\n :RANCHI JHARKHAND\nPlace of Birth                                 \nDate of Issue                                  :14/07/2011\nDate of Issue                                  :14/07/2011\nDate of Expiry                                :13/07/2021                                \nDate of Expiry                                :13/07/2021                                \nAvailability                                          : As per the requirement\nAvailability                                          : As per the requirement\n\n: H. E. C QQ  No:No: CD-440, Near Check Po\n\ns                                  : Married\nMarried\n\nss                       : H. E. C \n\nsh, Urdu, Arabic & Hindi\nArabic & Hindi\n\n CD-440, Near Check Post st DDhurwa Ranchi Jharkhand.\nhurwa Ranchi Jharkhand.\n\nDeclaration    \nDeclaration\nI Do Here By Confirm That The Information given In This Form Is True To Do The Best Of My Knowledge And Belief.\nI Do Here By Confirm That The Information given In This Form Is True To Do The Best Of My Knowledge And Belief.\n\nDate:11-2-2018\nDate:11-2-2018\n(TANZEER AHMAD SHADAB) \n(TANZEER AHMAD SHADAB) \n\n\x0c          \n                   \n                   \n\n\x0c' """
model.predict(text_to_summarize)

['Iam a highlymotivated and self-motivated individual with extensive background in allfacets of the business. I have been involved in various projects, both in storekeeping and in customer relations. I have worked as a Storekeeper/Warehousing Incharge for over 4 years.']

In [42]:
text_to_summarize="""summarize: b'SANJAYSINH  JADEJA\n\nVILLAGE\nTALUKA\nDIST\n\nMob.\nE-mail\n            \n\n:\n:\n:\n\n:\n:\n\nBEHIND POST OFFICE,\nMOTAKAPADA,\nDISA.\nBANASKATHA\nGUJARAT\n+91 9909545802\nsanjay4832@gmail.com\n \n\n  / sanjay4832@yahoo.co.in\n \n \n\n APPLICATION\n\nSub: Application for the vacant post stores in your esteemed \n\n .  \n Organization\n \n\n \n\nRespected Sir,\n\nI have come to know from some reliable sources that there is a vacancy in your reputed organization for\n\nabove position. I would like to apply my self for it.\n\nI have total 13 years experience store dept. of various construction companies. I will submit my credentials while\nreceiving your reply for attending the interview.\n\nI hope you will consider my application sympathically and looking forward for favorable, positive response at\nearliest convinced same as per requirement for the position.\n\nThanking you in anticipation.\n\nYours sincerely,\n\n(Jadeja Sanjaysinh S.)\n\n(Encl.: Curriculum Vitae)\n\n\x0cCURRICULUM VITAE\n\nSANJAYSINH  JADEJA\n\nVILLAGE\nTALUKA\nDIST\n\n:\n:\n:\n\nMob.\nE-mail\nPassport          :            N0649260 Validity 26.06.2025\n\n:\n:\n\n        \n\nCareer Objective:\n\nBEHIND POST OFFICE,\nMOTAKAPADA,\nDISA.\nBANASKATHA\nGUJARAT\n+91-9909545802                                                                                                       \nsanjay4832@gmail.com\n\nTo obtain a Sr. Store Officer Position with Carter\n\ns where I may be able to utilize my\nskills   in   managing   employees   and   store   operations   while   working   in   a   highly   stimulating   work\nenvironment.\n\n\xc2\x92s where I may be able to utilize my\n\nSanjaysinh Jadeja\nSukhdevsinh Jadeja\n5th May 1982\nMale\nMarried\n\n:\n:\n:\n:\n:\nEnglish, Hindi and Gujarati\n\nPersonal Information :-\n\n\xc2\x92s where I may be able to utilize my\n\ns Name\n\nName\nFather\nDate of Birth\nSex\nMarriage \nLanguage Proficiency :\n\nAcademic Qualification:-\n\nSr.\nNo.\na)\nb)\nc)\nd)\n\nAcademic\n\nQualification\n\nSSC\nHSC\nBBA\nMBA\n\nComputer Skills:-\n\nUniversity/ Board\n\nPercentage\n\nSIR AJITSINHJI GOVT. HIGHSCHOOL\nSIR AJITSINHJI GOVT. HIGHSCHOOL\n\nSIKKIM MANIPAL UNIVERSITY\nSIKKIM MANIPAL UNIVERSITY\n\nYear of\nPassing\n1996\n1998\n2012\n2014\n\n42.43\n50.83\n51.00\n58.15\n\n(1) Operating Systems\n(2)  Store\n\n: Windows XP, MS Office.\n: SAP (System Applications Products, ERP(ENTERPRISE\n\nRESOURCE PLANNING) PMS SYSTEM, TALLY9,\n\nWork Experience:-\n\n14 years Experiences in Stores\n\n     \nTotal experiences are in Store Dept. of Various Construction Companies which is taking contract of\nconstruction of Industrial Building, Factory, power projects, and Infrastructures projects.\n\nPresently Working: - KUNAL STRUTURE INDIA PVT LTD\nProject  \nPosition\nDuration\n\n    : Tawa Canal Project  (Harda \n    : Store Officer \n    : 13th Feb\xe2\x80\x99 2017 To Till Date.\n\n MP)\xc2\x96 MP)\n\n\x0cPreviously Working Company :-\n \n\nFrom\n\nTo\n\nName  of Company\n\nDuration\n\nLocation\n\n20/06/2015\n12/02/2012\n11/06/2007\n25/02/2005\n\n31/01/2017\n15/05/2015\n15/01/2012\n30/05/2007\n\nApco Infratech Pvt. Ltd.\nSuyog Infra (I) Pvt. Ltd.\n\nDesai Construction Pvt. Ltd.\n\nJMC Projects (I) Ltd.\n\n1 Yrs \n4 Yrs \n4 Yrs \n2 Yrs \n\n\xc2\x96 MP)\n\xc2\x96 MP)\n\xc2\x96 MP)\n\xc2\x96 MP)\n\n 8 Month\n 03 Month \n 07 Month     \n 03 Month\n\nUttar Pradesh\nGujarat & MH\nGujarat & MH\nHyderabad (A.P)\n\nPreviously Working Projects:-\n\nFrom\n\nTo\n\nName  of Company\n\nProject\n\nLocation\n\n20/06/2015\n\n31/01/2017\n\nApco Infratech Pvt. Ltd.\n\n15/05/2015\n\n15/01/2012\n\nSuyog Infra (I) Pvt. Ltd.\n\nDesai Construction Pvt. Ltd.\n\n14/02/2008\n\nDesai Construction Pvt. Ltd.\n\nGandak Canal\n\nProject\n\nFord India Project\n\nBombay Rayon\n\nFashion Ltd\nPower Project\n\nGorakhpur (UP)\n\nSanand(Guj)\n\nBoiser (MH)\n\nKovaya (GJ)\n\n30/05/2007\n\nJMC Projects (I) Ltd.\n\nWipro Technology\n\nHyderabad\n\n12/02/2012\n\n15/02/2008\n\n11/06/2007\n\n25/02/2005\n\nJob Profile :-\n\nResponsible for Maintain Books, Statements, Records as per ISO 9001.\nResponsible for managing all day to day stores related activities.\nCoordinating with site stores at different locations on a regular basis.\nMaterial Inward, outward management & Dispatch.\nReconciliation of materials.\nCo-ordination with purchase dept for all necessary documents.\nOverall inventory control.\nMaintain stock, supplies and inventories.\n\nSr.\nNo.\n1.\n    2.\n3.\n4.\n\nSr.\nNo.\n    1.\n\n2.\n\n3.\n\n4.\n\n5.\n\n1.\n2.\n3.\n4.\n5.\n6.\n7.\n8.\n\n\xef\x83\x98 Kamal Deep               :- +919473722439 (Apco -  Sr, Engineer)\n\xef\x83\x98 Pradeep Kumar        :- +919415816950 (Apco \xe2\x80\x93 HR & Admin)\n\nRs. 4.80 LPA + Food Facility + Bachelor Accommodation + Provident Fund  \n\nI hereby declare that the above provided information is correct to the best of my knowledge & \n\nReference:-\n\nCurrent Salary:-\n\nDeclaration:-\n\nbelief.\n\nDate:-\n\nPlace:-\n\n(Jadeja Sanjaysinh S.)\n\n\x0c'"""
model.predict(text_to_summarize)

['Aspiring and ambitious, I am looking for a position where I will be able to utilize my skills in managing multiple projects at one place.']

In [43]:
ls

 1073.pdf          sample_data/
 1088.pdf         'SUKANYA RAMESH -Senior Manager.docx'
 lightning_logs/  'Summary-dataset - db1.csv'
 outputs/         'Supriya Resume-1.pdf'


In [44]:
cd  outputs

/content/outputs


In [45]:
ls

simplet5-epoch-0-train-loss-4.0566-val-loss-3.5542/
simplet5-epoch-1-train-loss-3.5995-val-loss-3.4056/
simplet5-epoch-2-train-loss-3.3338-val-loss-3.3061/
simplet5-epoch-3-train-loss-3.1178-val-loss-3.232/
simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214/


In [53]:
!zip -r simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214.zip /content/outputs/simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214

  adding: content/outputs/simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214/ (stored 0%)
  adding: content/outputs/simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214/tokenizer_config.json (deflated 80%)
  adding: content/outputs/simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214/spiece.model (deflated 48%)
  adding: content/outputs/simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214/config.json (deflated 63%)
  adding: content/outputs/simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214/special_tokens_map.json (deflated 83%)
  adding: content/outputs/simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214/pytorch_model.bin (deflated 8%)
  adding: content/outputs/simplet5-epoch-4-train-loss-2.9161-val-loss-3.2214/tokenizer.json (deflated 74%)
